In [1]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from pymongo import MongoClient
from dotenv import load_dotenv
import time
from sleep_helper import SleepHelper


hello


In [2]:
# Load environment variables from .env file
load_dotenv()
MONGO_DB_URL = os.getenv("MONGO_DB_URL")



In [3]:
# Setup Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--always-enable-hdcp")



In [4]:
# Set path to chromedriver as per your configuration
webdriver_service = Service(ChromeDriverManager().install())


In [10]:
# Choose Chrome Browser
driver = webdriver.Chrome( options=chrome_options)


In [54]:
# !pip install selenium pygetwindow

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pygetwindow: filename=PyGetWindow-0.0.9-py3-none-any.whl size=11063 sha256=2c3365bfc3e286b896800f8895132e8289eb31ce1e31ea9d38f0d382b92e76d0
  Stored in directory: /Users/arifrahman/Library/Caches/pip/wheels/b3/39/81/34dd7a2eca5f885f1f6e2796761970daf66a2d98ac1904f5f4
  Created wheel for pyrect: filename=PyRect-0.2.0-py2.py3-none-any.whl size=11179 sha256=540058cd7d1377d7438861d7a260f4384710b0f21d156e1e39acc5dd4ad8f5f5
  Stored in directory: /Users/arifrahman/Library/Caches/pip/wheels/0b/1e/d7/0c74bd8f60b39c14d84e307398786002aa7ddc905927cc03c5
Successfully built pygetwindow pyrect


In [6]:
# Connect to MongoDB
client = MongoClient(MONGO_DB_URL)
db = client['job_miner']
collection = db['jobs_linkedin']



In [7]:
# Define the base URL and number of pages to scrape
base_url = 'https://www.linkedin.com/jobs/search/?keywords=data%20engineer&location=California%2C%20United%20States'
num_pages = 2



In [117]:
driver.get(base_url)


In [108]:
job_postings = driver.find_element(By.CLASS_NAME, 'jobs-search__results-list').find_elements(By.CLASS_NAME, 'base-card')


for job in job_postings[:2]:
    # print(job)
    print(job.find_element(By.CSS_SELECTOR, 'h3.base-search-card__title').text)
    print(job.find_element(By.CSS_SELECTOR, 'h4.base-search-card__subtitle').text)
    print(job.find_element(By.CSS_SELECTOR, 'span.job-search-card__location').text)
    print(job.find_element(By.CSS_SELECTOR, 'a.base-card__full-link').get_attribute('href'))
    job.find_element(By.CSS_SELECTOR, 'a.base-card__full-link').click()
    time.sleep(2)
    print(driver.find_element(By.CSS_SELECTOR, 'div.description__text').text)
    detail_sections = driver.find_elements(By.CSS_SELECTOR, 'li.description__job-criteria-item')
    for section in detail_sections:
        print(section.find_element(By.CSS_SELECTOR, 'h3.description__job-criteria-subheader').text)
        print(section.find_element(By.CSS_SELECTOR, 'span.description__job-criteria-text').text)
        
        
        
            

Data Engineer (L5)- Business Intelligence
Netflix
Los Gatos, CA
https://www.linkedin.com/jobs/view/data-engineer-l5-business-intelligence-at-netflix-3939132186?position=1&pageNum=0&refId=99pOCydF4uUGUe6nc7qcQQ%3D%3D&trackingId=g5pRTEpTEURAm3492ZDABw%3D%3D&trk=public_jobs_jserp-result_search-card
At Netflix, our mission is to entertain the world. With 200+ million paid members in over 190 countries on millions of devices; enjoying TV series, documentaries, and feature films across a wide variety of genres and languages - Netflix is reinventing entertainment from end to end. We are revolutionizing how shows and movies are produced, pushing technological boundaries to efficiently deliver streaming video at a massive scale over the internet, and continuously improving the end-to-end user experience with Netflix across their member journey.

We pride ourselves on using data to inform our decision-making as we work towards our mission. This requires curating data across various domains, such

In [92]:
print(driver.find_element(By.CSS_SELECTOR, 'div.compensation__salary-range').find_element(By.CSS_SELECTOR, 'h3.compensation__heading').text)
print(driver.find_element(By.CSS_SELECTOR, 'div.compensation__salary-range').find_element(By.CSS_SELECTOR, 'div.compensation__salary').text)

Base pay range
$170,000.00/yr - $720,000.00/yr


In [11]:
import json

for page in range(num_pages):
    url = f'{base_url}&start={page * 25}'
    driver.get(url)
    SleepHelper.random_sleep(12, 2)  # wait for the page to load

    # Find all job postings
    job_postings = driver.find_element(By.CLASS_NAME, 'jobs-search__results-list').find_elements(By.CLASS_NAME, 'base-card')

    # Extract information from each job posting
    for job in job_postings:
        try:
            # Extract job title
            title_elem = job.find_element(By.CSS_SELECTOR, 'h3.base-search-card__title')
            title = title_elem.text.strip() if title_elem else 'N/A'

            # Extract company name    
            company_elem = job.find_element(By.CSS_SELECTOR, 'h4.base-search-card__subtitle')
            company = company_elem.text.strip() if company_elem else 'N/A'

            # Extract job location
            location_elem = job.find_element(By.CSS_SELECTOR, 'span.job-search-card__location')
            location = location_elem.text.strip() if location_elem else 'N/A'

            # Extract job link
            joblink_elem = job.find_element(By.CSS_SELECTOR, 'a.base-card__full-link')
            joblink = joblink_elem.get_attribute('href').strip() if joblink_elem else 'N/A'

            # Click on the job link to get detailed information
            joblink_elem.click()
            SleepHelper.random_sleep(12, 2)  # wait for the job detail page to load

            # # Switch to the job detail window
            # driver.switch_to.window(driver.window_handles[1])

            # Extract job description
            description_elem = driver.find_element(By.CSS_SELECTOR, 'div.description__text')
            description = description_elem.text.strip() if description_elem else 'N/A'

            # Extract job details (e.g., salary, employment type, etc.)
            detail_sections = driver.find_elements(By.CSS_SELECTOR, 'li.description__job-criteria-item')
            details = {}
            
            key_elem = driver.find_element(By.CSS_SELECTOR, 'div.compensation__salary-range').find_element(By.CSS_SELECTOR, 'h3.compensation__heading')
            value_elem = driver.find_element(By.CSS_SELECTOR, 'div.compensation__salary-range').find_element(By.CSS_SELECTOR, 'div.compensation__salary')
            key = key_elem.text.strip() if key_elem else 'N/A'
            value = value_elem.text.strip() if value_elem else 'N/A'
            details[key] = value
            
            for section in detail_sections:
                key_elem = section.find_element(By.CSS_SELECTOR, 'h3.description__job-criteria-subheader')
                value_elem = section.find_element(By.CSS_SELECTOR, 'span.description__job-criteria-text')
                key = key_elem.text.strip() if key_elem else 'N/A'
                value = value_elem.text.strip() if value_elem else 'N/A'
                details[key] = value

            # Print job information
            # print(f'Job Title: {title}')
            # print(f'Company: {company}')
            # print(f'Location: {location}')
            # print(f'Job Link: {joblink}')
            # print(f'Details: {details}')
            # print(f'Description: {description}')
            # print('-' * 40)

            # Create a dictionary to store the job data
            job_data = {
                'title': title,
                'company': company,
                'location': location,
                'job_link': joblink,
                'details': details,
                'description': description
            }
            
            print(json.dumps(job_data, indent=4))
            print('-' * 40)
            
            # Insert the job data into MongoDB
            collection.insert_one(job_data)

            # # Close the job detail window and switch back to the main window
            # driver.close()
            # driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(f"Error processing job: {e}")
            continue



sleep time: 11.552535027106208
sleep time: 15.38186519102215
{
    "title": "Data Engineer (L5)- Business Intelligence",
    "company": "Netflix",
    "location": "Los Gatos, CA",
    "job_link": "https://www.linkedin.com/jobs/view/data-engineer-l5-business-intelligence-at-netflix-3939132186?position=1&pageNum=0&refId=uUdo4%2Bsi7QYYe19JoWPYiA%3D%3D&trackingId=hfA6tzcW7sgBwLvNxiJhTA%3D%3D&trk=public_jobs_jserp-result_search-card",
    "details": {
        "Base pay range": "$100,000.00/yr - $720,000.00/yr",
        "Seniority level": "Not Applicable",
        "Employment type": "Full-time",
        "Job function": "Information Technology",
        "Industries": "Entertainment Providers, Technology, Information and Internet, and Movies, Videos, and Sound"
    },
    "description": "At Netflix, our mission is to entertain the world. With 200+ million paid members in over 190 countries on millions of devices; enjoying TV series, documentaries, and feature films across a wide variety of gen

KeyboardInterrupt: 

In [ ]:
# Close the browser
driver.quit()
